In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from docplex.mp.model import Model

In [2]:
macrotick = 100
sync_error = 0
time_out = 4 * 60 * 60

NUM_FLOW = 50
DATA_NAME = "single0"
TOPO_NAME = "2"

task = pd.read_csv("../../dac_data/%s.csv"%DATA_NAME)[:50]
network = pd.read_csv("../../dac_data/%s_topology.csv"%TOPO_NAME)
for col in ['size','period','deadline','jitter']:
    task[col] = np.ceil(task[col] / macrotick).astype(int)
for col in ['t_proc','t_prop']:
    network[col] = np.ceil(network[col] / macrotick).astype(int)
    
nodes = list(network['link'].apply(lambda x:eval(x)[0])) + \
    list(network['link'].apply(lambda x:eval(x)[1]))
NODE_SET = list(set(nodes))
ES_set = [x for x in NODE_SET if nodes.count(x) == 2]
SW_set = list(set(NODE_SET) - set(ES_set))
LCM = np.lcm.reduce(task['period'])
net = np.zeros(shape = (max(NODE_SET) + 1, max(NODE_SET) + 1))

## 1. Model

In [3]:
m = Model(name='RTCAS2018', log_output=True)

network model

\begin{array}{cl}
\mathcal{E}=\{0,1,2, \ldots\} \subset \mathbb{N} & \text { set of edges } \\
\mathcal{V}=\{0,1,2, \ldots\} \subset \mathbb{N} & \text { set of vertices } \\
\mathbf{A}_{E E} \in\{0,1\}|\mathcal{E}| \times|\mathcal{E}| & \text { sparse edge-edge adjacency matrix } \\
\mathbf{B}_{V E} \in\{-1,0,1\}|\mathcal{V}| \times|\mathcal{E}| & \text { sparse vertex-edge incidence matrix } \\
D \in \mathbb{N} & \text { time it takes for crossing over a vertex } \\
\end{array}

In [4]:
net_var = {}

for _, row in network.iterrows():
    net_var.setdefault(eval(row['link'])[0], {})
    net_var[eval(row['link'])[0]]['msd'] = int(row['t_proc'])
    net[eval(row['link'])[0], eval(row['link'])[1]] = 1

## Create mapping from Link to index
link_to_index = {}
index_to_link = {}

counter = 0
for _, row in network.iterrows():
    link = row['link']
    link_to_index[link] = counter
    index_to_link[counter] = link
    counter += 1

In [5]:
A = np.zeros(shape = (len(link_to_index), len(link_to_index)))
for a in index_to_link:
    for b in index_to_link:
        link_a, link_b = index_to_link[a], index_to_link[b]
        if eval(link_a)[1] == eval(link_b)[0]:
            A[a][b] = 1

B = np.zeros(shape=(max(NODE_SET) + 1, len(link_to_index)))
for v in NODE_SET:
    for e in index_to_link:
        link = eval(index_to_link[e])
        if link[0] == v:
             B[v][e] = 1
        elif link[1] == v:
             B[v][e] = -1

$$\mathbf{u}[f][e]=\left\{\begin{array}{l}
1 \rightarrow \text { flow } f \text { uses edge } e \\
0 \rightarrow \text { flow } f \text { does not use edge } e
\end{array}\right.$$

In [6]:
u = m.binary_var_matrix(len(task), len(link_to_index))

task model

\begin{array}{cl}
\hline \mathcal{F} \subset \mathbb{N} & \text { set of flow indices } \\
\mathbf{o}_{F} \in \mathcal{V}^{|\mathcal{F}|} & \text { vector of flow origins: } \mathbf{o}_{F}[f]=v, \text { iff flow } f \in \mathcal{F} \\
\mathbf{d}_{F} \in \mathcal{V}^{|\mathcal{F}|} & \text { starts at vertex } v \in \mathcal{V} \\
& \text { vector of flow destinations: } \mathbf{d}_{F}[f]=v, \text { iff flow } f \in \\
\mathbf{p}_{F} \in \mathbb{N}^{|\mathcal{F}|} & \mathcal{F} \text { ends at vertex } v \in \mathcal{V} \\
& \text { vector of flow period: } \mathbf{p}_{F}[f]=p_{f}, \text { iff flow } f \in \mathcal{F} \\
\mathbf{r}_{F} \in \mathbb{N}^{|\mathcal{F}|} & \text { has a period of } p_{f} \in \mathbb{N} \\
& \text { vector of required transmission durations of flows } \\
& \text { per period: } \mathbf{r}_{F}[f]=r_{f}, \text { iff flow } f \in \mathcal{F} \text { needs a } \\
& \text { reservation of } r_{f} \in \mathbb{N} \\
\mathbf{l}_{F} \in \mathbb{N}^{|\mathcal{F}|} & \text { vector of maximally allowed end-to-end deadline for } \\
& \text { flows: } \mathbf{l}_{F}[f]=t_{f}, \text { iff flow } f \in \mathcal{F} \text { has a maximal } \\
& \text { end-to-end deadline of } t_{f} \in \mathbb{N} \\
h=\operatorname{lcm}\left(\mathbf{p}_{F}\right) & \text { least common multiple of all individual flow periods } \\
\hline
\end{array}

In [7]:
task_attr = {}

for k, row in task.iterrows():
#     task_var.setdefault(k, {})
    task_attr.setdefault(k, {})
    
    
    ## f_k
    task_attr[k]['o'] = int(row['src'])
    task_attr[k]['d'] = int(eval(row['dst'])[0])
    task_attr[k]['p'] = int(row['period'])
    task_attr[k]['r'] = int(row['size'] * 8)
    task_attr[k]['l'] = int(row['deadline'])
    
h = int(np.lcm.reduce(task['period']))

$$\mathbf{t} \in \mathbb{N}^{|\mathcal{F}| \times|\mathcal{V}|} \text { with } 0 \leq \mathbf{t}[f][*] \leq \mathbf{p}_{F}[f]-\mathbf{r}_{F}[f]$$

In [8]:
t = m.integer_var_matrix(len(task), len(link_to_index))
for k in range(len(task)):
    for e in index_to_link:
        m.add_constraint(0 <= t[k, e])
        m.add_constraint(t[k, e] <= task_attr[k]['p'] - task_attr[k]['r'])

## 2. Constraints

### 2.1 Scheduling Constraints

Scheduling Constraints : No overlap

$\forall f_{1} \in \mathcal{F}, f_{2} \in \mathcal{F}: f_{1} \neq f_{2}, e \in \mathcal{E}:$ $\forall a \in \mathcal{A}, b \in \mathcal{B}:$ $\quad$ if $\left(\mathbf{u}\left[f_{1}\right][e]+\mathbf{u}\left[f_{2}\right][e] \geq 2\right)$ then $\quad\left(\mathbf{t}\left[f_{1}\right][e]+a \cdot \mathbf{p}_{F}\left[f_{1}\right] \geq \mathbf{t}_{F}\left[f_{2}\right][e]+b \cdot \mathbf{p}_{F}\left[f_{2}\right]+\mathbf{r}_{F}\left[f_{2}\right]\right.$ $\left.\quad \operatorname{or} \mathbf{t}\left[f_{2}\right][e]+b \cdot \mathbf{p}_{F}\left[f_{2}\right] \geq \mathbf{t}_{F}\left[f_{1}\right][e]+a \cdot \mathbf{p}_{F}\left[f_{1}\right]+\mathbf{r}_{F}\left[f_{1}\right]\right)$
with
$$
\begin{aligned}
&\mathcal{A}=\left\{a \in \mathbb{N}: 0 \leq a \leq \frac{h}{\mathbf{p}_{F}\left[f_{1}\right]}\right\} \\
&\mathcal{B}=\left\{b \in \mathbb{N}: 0 \leq b \leq \frac{h}{\mathbf{p}_{F}\left[f_{2}\right]}\right\}
\end{aligned}
$$

In [9]:
for f1, f2 in tqdm([(f1, f2) for f1 in task_attr for f2 in task_attr if f1 < f2]):
    p1, p2 = task_attr[f1]['p'], task_attr[f2]['p']
    r1, r2 = task_attr[f1]['r'], task_attr[f2]['r']
    for e in index_to_link:
        _lcm = np.lcm(p1, p2)
        for a, b in [(a,b) for a in range(int(_lcm / p1)) for b in range(int(_lcm / p2))]:  
            m.add_constraint(
                    m.logical_or(
                        u[f1,e] == 0,
                        u[f2,e] == 0,
                        t[f1,e] + a * p1 >= t[f2,e] + b * p2 + r2 + 1,
                        t[f2,e] + b * p2 >= t[f1,e] + a * p1 + r1 + 1
                    ) == 1             
                )
            
            
#             m.addConstr(
#                 (u[f1][e] + u[f2][e] >= 2) >>
#                 gp.or_((t[f1][e] + a * p1 >= t[f2][e] + b * p2 + r2),
#                 (t[f2][e] + b * p2 >= t[f1][e] + a * p1 + r1))
#             )

100%|██████████| 1225/1225 [00:29<00:00, 40.93it/s]


### 2. Routing constraints

$$\forall f \in \mathcal{F}: \sum_{e \in \mathcal{E}} \mathbf{B}_{V E}\left[\mathbf{o}_{F}[f]\right][e] \cdot \mathbf{u}[f][e]=-1$$

In [10]:
## This formular in the paper is wrong
for f in task_attr:
    m.add_constraint(m.sum([B[task_attr[f]['o']][e] * u[f, e] for e in index_to_link]) == 1)
    ## Only 1 link from ES in path
    m.add_constraint(m.sum(u[f, e] for e in index_to_link if eval(index_to_link[e])[0] in ES_set) == 1)
    ## m.add_constraint(m.sum([B[task_attr[f]['o']][e] * u[f, e] for e in index_to_link if B[task_attr[f]['o']][e] == 1]) == 1)

$$\forall f \in \mathcal{F}: \sum_{e \in \mathcal{E}} \mathbf{B}_{V E}\left[\mathbf{d}_{F}[f]\right][e] \cdot \mathbf{u}[f][e]=1$$

In [11]:
## This formular in the paper is wrong
for f in task_attr:
    m.add_constraint(m.sum([B[task_attr[f]['d']][e] * u[f, e] for e in index_to_link]) == -1)
    ## Only 1 link into ES in path
    m.add_constraint(m.sum(u[f, e] for e in index_to_link if eval(index_to_link[e])[1] in ES_set) == 1)
    ## m.add_constraint(m.sum([B[task_attr[f]['d']][e] * u[f, e] for e in index_to_link if B[task_attr[f]['d']][e] == -1]) == -1)

\begin{gathered}
\forall f \in \mathcal{F}: \sum_{\mathbf{B}_{V E}[v][e]=1} \mathbf{B}_{V E}[v][e] \cdot \mathbf{u}[f][e] \\
=-\sum_{\mathbf{B}_{V E}[v][e]=-1} \mathbf{B}_{V E}[v][e] \cdot \mathbf{u}[f][e] \\
\text { with } \quad v \in \mathcal{V} \backslash\left\{\mathbf{o}_{F}[f], \mathbf{d}_{F}[f]\right\}, e \in \mathcal{E}
\end{gathered}

In [12]:
for f in task_attr:
    for v in SW_set:
        m.add_constraint(
            m.sum(
                B[v][e] * u[f, e] 
                for e in index_to_link if B[v][e] == 1
            )
            +
            m.sum(
                B[v][e] * u[f, e] 
                for e in index_to_link if B[v][e] == -1
            )
            == 0
    )

### Constraints Linking Routing and Scheduling Constraints

$\forall e_{p} \in \mathcal{E}, \forall e_{n} \in \mathcal{E}: e_{p} \neq e_{n}$ and $\mathbf{A}_{E E}\left[e_{p}\right]\left[e_{n}\right]=1, \forall f \in \mathcal{F}:$
if $\left(\mathbf{u}[f]\left[e_{p}\right]+\mathbf{u}[f]\left[e_{n}\right] \geq 2\right)$ then
$\left(\mathbf{t}[f]\left[e_{n}\right]=\mathbf{t}[f]\left[e_{p}\right]+D\right.$
$\left.\operatorname{or} \mathbf{t}[f]\left[e_{n}\right]+\mathbf{p}_{F}[f]=\mathbf{t}[f]\left[e_{p}\right]+D\right)$

In [14]:
for ep, en in[(ep, en) for ep in index_to_link for en in index_to_link if 
               ep != en and A[ep][en] == 1]:
    
    for f in task_attr:
        m.add_constraint(
            m.logical_or(
                u[f, ep] == 0,
                u[f, en] == 0,
                t[f, en] == t[f, ep] + net_var[eval(index_to_link[ep])[0]]['msd'] + task_attr[f]['r'],
                t[f, en] + task_attr[f]['p'] == t[f, ep] + net_var[eval(index_to_link[ep])[0]]['msd'] + task_attr[f]['r']
            ) == 1
        )

### end-to-end delay

$$\forall f \in \sum_{e \in \mathcal{E}} D \cdot \mathbf{u}[f][e] \leq l_{F}[f]$$

In [15]:
for f in task_attr:
    m.add_constraint(
         (net_var[eval(index_to_link[e])[0]]['msd'] + task_attr[f]['r']) * m.sum(u[f, e] for e in index_to_link) <= task_attr[f]['l']
    )

In [16]:
result = m.solve()

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
Presolve has eliminated 99200 rows and 99802 columns...
Tried aggregator 2 times.
MIP Presolve eliminated 336337 rows and 159766 columns.
MIP Presolve modified 122858 coefficients.
Aggregator did 189703 substitutions.
Reduced MIP has 169048 rows, 151827 columns, and 459795 nonzeros.
Reduced MIP has 110018 binaries, 2000 generals, 0 SOSs, and 175636 indicators.
Presolve time = 2.02 sec. (76867.28 ticks)
Probing fixed 2 vars, tightened 0 bounds.
Probing changed sense of 291 constraints.
Probing time = 6.06 sec. (198.39 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 401 rows and 293 columns.
MIP Presolve modified 291 coefficients.
Reduced MIP has 168647 rows, 151534 columns, and 458767 nonzeros.
Reduced MIP has 109725 binaries, 41809 generals, 0 SOSs, and 175636 indicators.
Presolve time = 1.25 sec. (573.96 ticks)
Probing time = 1.53 sec. (61.29 ticks)


## Output Schedule

In [ ]:
## GCL
GCL = []

for i in task_attr:
    for e_i, e in [(e, index_to_link[e]) for e in index_to_link if result.get_value(u[i, e]) == 1]:
        start = result.get_value(t[i, e_i])
        end = start + task_attr[i]['r']
        queue = 0
        tt = task_attr[i]['p']
        for k in range(int(LCM / tt)):
            GCL.append(
                [eval(e), queue, (start + k * tt) * macrotick, (end + k * tt) * macrotick, LCM * macrotick]
            )

In [ ]:
## Offset
OFFSET = []
for i in task_attr:
    start_index = np.where(B[task_attr[i]['o']] == 1)[0]
    start_index = [x for x in start_index if result.get_value(u[i, x]) == 1][0]
    start_link = index_to_link[start_index]
    offset = result.get_value(t[i, start_index])
    OFFSET.append(
        [i, 0, (task.loc[i,'period'] - offset) * macrotick]
    )    

In [ ]:
ROUTE = []
for i in task_attr:
    path = [index_to_link[e] for e in index_to_link if result.get_value(u[i, e]) == 1]
    for link in path:
        ROUTE.append([i, eval(link)])

In [ ]:
QUEUE = []
for i in task_attr:
    for e in [index_to_link[e] for e in index_to_link if result.get_value(u[i, e]) == 1]:
        QUEUE.append([i, 0, eval(e), 0])

In [ ]:
GCL = pd.DataFrame(GCL)
GCL.columns = ["link", "queue", "start", "end", "cycle"]
GCL.to_csv("RTCSA2018-%s-GCL.csv"%(DATA_NAME + '-' + TOPO_NAME), index=False)

OFFSET = pd.DataFrame(OFFSET)
OFFSET.columns = ['id', 'ins_id', 'offset']
OFFSET.to_csv("RTCSA2018-%s-OFFSET.csv"%(DATA_NAME + '-' + TOPO_NAME), index=False)

ROUTE = pd.DataFrame(ROUTE)
ROUTE.columns = ['id', 'link']
ROUTE.to_csv("RTCSA2018-%s-ROUTE.csv"%(DATA_NAME + '-' + TOPO_NAME), index=False)

QUEUE = pd.DataFrame(QUEUE)
QUEUE.columns = ['id','ins_id','link','queue']
QUEUE.to_csv("RTCSA2018-%s-QUEUE.csv"%(DATA_NAME + '-' + TOPO_NAME), index=False)